In [1]:
import pandas as pd
from dateutil import parser
import pdfminer
from pdfminer.high_level import extract_text
import glob
import nltk
from string import punctuation
from nltk.corpus import stopwords
import re
from nltk.tag import pos_tag
from names_dataset import NameDataset
nd = NameDataset()
import spacy #numpy needs to be <1.20
nlp = spacy.load("en_core_web_sm")
import os
import pdfplumber
import pandas as pd
import datefinder
from datetime import datetime
from thefuzz import fuzz
# from pyresparser import ResumeParser
import numpy as np

C:\Users\rtd91\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
pyres_df = pd.read_csv(r"C:/Users/rtd91/code/nlp-coe/resume-parser/pyres_df.csv")
gt_df = pd.read_csv(r'C:/Users/rtd91/Data/resume_samples_output_data/Ground Truth-v3.csv', index_col=False)

In [3]:
pyres_df

,Unnamed: 0,name,email,mobile_number,skills,college_name,degree,designation,experience,company_names,no_of_pages,total_experience,text,index
0,0,Karthik Ramanarayana,karthikr2194@gmail.com,292-1151,"['Reports', 'Analyze', 'Debugging', 'Word', 'H...",NaN,NaN,NaN,"['Data Intern, Index Analytics LLC, Baltimore,...",NaN,1,0.00,"Karthik Ramanarayana \n2001 Eastern Avenue, Ba...",721091408_phonescreening.pdf
1,1,HARSH PUNDIR,hpundir@umd.edu,240.423.5453,"['Ai', 'R', 'Reports', 'Analyze', 'Payments', ...",NaN,NaN,NaN,['■ Restaurant Ranking System: Prescriptive a...,NaN,1,0.00,"HARSH PUNDIR \n3425 Tulane Dr. Hyattsville,...",821101104_phonescreening.pdf
2,2,TX Master,tirth2410@gmail.com,469-370-9437,"['Calculus', 'Analyze', 'R', 'Metrics', 'Adobe...",NaN,NaN,NaN,"['Web Analyst, The University of Texas at Dall...",NaN,1,2.83,"\n\n \n\nTIRTH PATEL \nHolbrook, NY | tirth24...",821101136_phonescreening.pdf
3,3,Akanksha Bapna,NaN,NaN,"['Ibm', 'Crm', 'Ai', 'R', 'Research', 'Present...",NaN,"Bachelors in Technology, Chemical Engineering",NaN,"['CollegePark, MD, USA', 'Jun 2021 – Aug 2021'...",NaN,2,0.17,Akanksha Bapna\nabapna.com ● Email ● Personal ...,821110120.pdf
4,4,Mythri Partha,mythripartha8@gmail.com,725-7080,"['Access', 'Analyze', 'Email', 'Python', 'Prog...",NaN,NaN,NaN,"['InventXYZ, Data Engineering Intern', 'June 2...",NaN,1,1.00,"Mythri Partha \nHouston, TX | Phone: (281) 725...",821110440_Mythri.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,176,Dong yd2578@columbia.edu·(319,yd2578@columbia.edu·(319),512-0421,"['Ibm', 'Analyze', 'R', 'Reports', 'Sqlalchemy...",NaN,M.S. in Computer and Information Science,NaN,['Streaming Data Retrieval System ...,['Oracle'],1,0.00,\n\n \n\n \n\n \n\n \n\nYunhan (Zoe) Dong \n\...,YunhanDong.pdf
177,177,Jersey City,zzhan67@stevens.edu,531-8942,"['R', 'Statistics', 'System', 'Python', 'Progr...",NaN,Master of Science in Data Science) GPA,['Data Engineer'],"['Utegration LLC, Houston, TX', 'Data Analytic...",NaN,1,0.00,"\n\nJersey City, NJ ...",ZhengyangZhang.pdf
178,178,Zhuohan Zhang,zz2751@columbia.edu,763-4795,"['R', 'Research', 'Economics', 'Sas', 'Statist...",NaN,"Master of Arts, Statistics","['Data Scientist Consultant ', 'Data A...",['Project on Covid Social Media Data ...,NaN,1,0.75,Zhuohan Zhang \n (617) 763-4795 | zz2751@colum...,ZhuohanZhang.pdf
179,179,Zixiao Huang,zixiao.huang.74@gmail.com,615-977-2160,"['Calculus', 'Reports', 'R', 'Metrics', 'Stati...",NaN,Master of Science in Analytics,NaN,"['Stealth Mode Startup, Deep Learning Innovati...",NaN,1,1.25,Zixiao Huang \n\nzixiao.huang.74@gmail.com | 6...,ZixiaoHuang.pdf


In [4]:
compared_df = pd.merge(pyres_df, gt_df, how = 'left', on = 'index')

In [4]:
compared_df.shape

(181, 26)

In [5]:
compared_df.columns

Index(['Unnamed: 0', 'name', 'email_x', 'mobile_number', 'skills',
       'college_name', 'degree', 'designation', 'experience', 'company_names',
       'no_of_pages', 'total_experience', 'text', 'index', 'full_name',
       'phd_school', 'phd_degree', 'phd_graduation_date', 'master_school',
       'master_degree', 'master_graduation_date', 'bachelor_school',
       'bachelor_degree', 'bachelor_graduation_date', 'email_y', 'phone'],
      dtype='object')

In [6]:
def unlist(x):
    if isinstance(x, list) and len(x) != 0:
        for n in x:
            return n
    if x is None:
        return ''
    else:
        return ''

# -------------------- NAME -----------------------------------------------------------

compared_df['full_name_true'] = compared_df['full_name'].fillna('')
compared_df['full_name_true'].replace(to_replace=[None], value='', inplace=True)
compared_df['full_name_true'] = compared_df.full_name_true.apply(lambda x: str(x).capitalize())


compared_df['name_ref'] = compared_df.name.apply(lambda x:unlist(x))
compared_df['name_ref'] = compared_df['name'].fillna('')
compared_df['name_ref'].replace(to_replace=[None], value='', inplace=True)
compared_df['name_ref'] = compared_df.name.apply(lambda x:str(x).capitalize())

"""
compared_df.columns
compared_df['unlisted_name_ner'] = compared_df.name_ner.apply(lambda x:unlist(x))
compared_df['unlisted_name_ner'] = compared_df['unlisted_name_ner'].fillna('')
compared_df['unlisted_name_ner'].replace(to_replace=[None], value='', inplace=True)
compared_df['unlisted_name_ner'] = compared_df.unlisted_name_ner.apply(lambda x:str(x).capitalize())

compared_df['unlisted_name_pos'] = compared_df.name_pos.apply(lambda x:unlist(x))
compared_df['unlisted_name_pos'] = compared_df['unlisted_name_pos'].fillna('')
compared_df['unlisted_name_pos'].replace(to_replace=[None], value='', inplace=True)
compared_df['unlisted_name_pos'] = compared_df.unlisted_name_pos.apply(lambda x:str(x).capitalize())

compared_df['unlisted_name_ref'] = compared_df.name_ref.apply(lambda x:unlist(x))
compared_df['unlisted_name_ref'] = compared_df['unlisted_name_ref'].fillna('')
compared_df['unlisted_name_ref'].replace(to_replace=[None], value='', inplace=True)
compared_df['unlisted_name_ref'] = compared_df.unlisted_name_ref.apply(lambda x:str(x).capitalize())"""

"\ncompared_df.columns\ncompared_df['unlisted_name_ner'] = compared_df.name_ner.apply(lambda x:unlist(x))\ncompared_df['unlisted_name_ner'] = compared_df['unlisted_name_ner'].fillna('')\ncompared_df['unlisted_name_ner'].replace(to_replace=[None], value='', inplace=True)\ncompared_df['unlisted_name_ner'] = compared_df.unlisted_name_ner.apply(lambda x:str(x).capitalize())\n\ncompared_df['unlisted_name_pos'] = compared_df.name_pos.apply(lambda x:unlist(x))\ncompared_df['unlisted_name_pos'] = compared_df['unlisted_name_pos'].fillna('')\ncompared_df['unlisted_name_pos'].replace(to_replace=[None], value='', inplace=True)\ncompared_df['unlisted_name_pos'] = compared_df.unlisted_name_pos.apply(lambda x:str(x).capitalize())\n\ncompared_df['unlisted_name_ref'] = compared_df.name_ref.apply(lambda x:unlist(x))\ncompared_df['unlisted_name_ref'] = compared_df['unlisted_name_ref'].fillna('')\ncompared_df['unlisted_name_ref'].replace(to_replace=[None], value='', inplace=True)\ncompared_df['unlisted_na

In [7]:
comp_name1 = compared_df.apply(lambda x: fuzz.ratio(x['full_name_true'], x['name_ref']), axis = 1)
np.mean(comp_name1) #73.14

73.13812154696133

In [8]:
# -------------------- PHONE NUMBER -----------------------------------------------------

# phone_number, phone
from string import punctuation

def clean_phone_number(x):
    if isinstance(x, str):
        return_x = ''.join([p for p in x if p not in punctuation]).replace(" ", "")
        #print(return_x)
        if len(return_x) != 10 and return_x.startswith('1'):
            return return_x[1:]
        else:
            return return_x
    if not isinstance(x, str):
        return ''

compared_df['number_pred'] = compared_df.mobile_number.apply(lambda x:clean_phone_number(x))
compared_df['phone_ref'] = compared_df.phone.apply(lambda x:clean_phone_number(x))


comp_phone = compared_df.apply(lambda x: fuzz.ratio(x['number_pred'], x['phone_ref']), axis = 1)
np.mean(comp_phone) #83.46

83.45856353591161

In [17]:
# --------------- EMAIL ----------------------

def unlist(x):
    if isinstance(x, list) and len(x) != 0:
        for n in x:
            return n
    if x is None:
        return ''
    else:
        return ''

compared_df['email_pref'] = compared_df.email_x.apply(lambda x:str(x).lower().replace(" ", ""))
compared_df['email_true'] = compared_df.email_y.apply(lambda x: str(x).replace("['", '').replace("']", '').replace("[", '').replace("]", ''))
comp_email = compared_df.apply(lambda x: fuzz.ratio(x['email_pref'], x['email_true']), axis = 1)
np.mean(comp_email) #97.82

97.81767955801105

In [ ]:
# ---------------------------- EDUCATION (to be cleaned) ---------------------------------------------------------------------------------------
compared_df = pd.merge(df, gt_df, how = 'left', on = 'index')
compared_df.columns

compared_df.master_degree_x
compared_df.master_degree2

compared_df.master_degree_y
compared_df.Master_Degree_2
compared_df.Master_Degree_3

def clean_degree_set(set_x):
    if not isinstance(set_x, set):
        return ''
    else:
        x = unlist(list(set_x))
        print(x)
        if isinstance(x, str):
            return_x = ''.join([p for p in x if p not in punctuation])
            print(return_x)
            return re.sub(' +', ' ', return_x).lower()

def clean_degree_string(string_x):

    if isinstance(string_x, str):
        return_x = ''.join([p for p in string_x if p not in punctuation])
        print(return_x)
        return re.sub(' +', ' ', return_x).lower()
    else:
        return ''


compared_df['cleaned_master_degree_pred2'][196]
compared_df['cleaned_master_degree_pred1'][196]
compared_df['cleaned_master_degree_true1'][196]
compared_df.master_degree_x[196]
compared_df.master_degree2[196]
compared_df['cleaned_master_degree_true1'][196]

compared_df['cleaned_master_degree_pred1'] = compared_df.master_degree_x.apply(lambda x:clean_degree_set(x))
compared_df['cleaned_master_degree_pred2'] = compared_df.master_degree2.apply(lambda x:clean_degree_string(x))

compared_df['cleaned_master_degree_true1'] = compared_df.master_degree_y.apply(lambda x:clean_degree_string(x))
compared_df['cleaned_master_degree_true2'] = compared_df.Master_Degree_2.apply(lambda x:clean_degree_string(x))
compared_df['cleaned_master_degree_true3'] = compared_df.Master_Degree_3.apply(lambda x:clean_degree_string(x))

compared_df.Master_Degree_2[compared_df.Master_Degree_2.notnull()] #15
compared_df['cleaned_master_degree_true1'][15]
compared_df['cleaned_master_degree_true2'][15]

compared_df['cleaned_master_degree_pred2'][15]
compared_df['cleaned_master_degree_pred1'][15]

fuzz.ratio(compared_df['cleaned_master_degree_pred2'][15], compared_df['cleaned_master_degree_true1'][15])
fuzz.ratio(compared_df['cleaned_master_degree_true2'][15], compared_df['cleaned_master_degree_pred2'][15])

fuzz.token_sort_ratio(compared_df['cleaned_master_degree_pred2'][15], compared_df['cleaned_master_degree_true1'][15])
fuzz.token_sort_ratio(compared_df['cleaned_master_degree_true2'][15], compared_df['cleaned_master_degree_pred2'][15])

fuzz.token_set_ratio(compared_df['cleaned_master_degree_true1'][15], compared_df['cleaned_master_degree_pred2'][15])
fuzz.token_set_ratio(compared_df['cleaned_master_degree_true2'][15], compared_df['cleaned_master_degree_pred2'][15])

# ---------------------- compare to other packages -------------------------------------------------------------------
from resume_parser import resumeparse
# data = resumeparse.read_file('/path/to/resume/file')
# data = ResumeParser(file_path).get_extracted_data()

# temp_person = []
# tagged = nlp(sentence_txt[0:200]) #Comment: assume names exist in the top section
#     for word in tagged.ents:
#         if word.label_ == "PERSON":
#             temp_person.append(word.text)